In [ ]:
import asyncio
from telegram import Update, ReplyKeyboardMarkup
from telegram.ext import (
    ApplicationBuilder, CommandHandler, MessageHandler, ConversationHandler,
    ContextTypes, filters
)
import pdfplumber
import re

def extract_courses_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages)
    blocks = {'mandatory': [], 'elective': []}
    current = None
    lines = text.split('\n')
    for line in lines:
        l = line.strip()
        if re.match(r'Обязатель.*дисциплин', l, re.I):
            current = 'mandatory'
            continue
        if re.match(r'Пул выборных дисциплин', l, re.I):
            current = 'elective'
            continue
        if current:
            if 3 < len(l) < 70 and not re.match(r'^\d+[\s\.]*$', l) and len(l.split()) < 12:
                blocks[current].append(l)
    blocks['mandatory'] = list(dict.fromkeys(blocks['mandatory']))
    blocks['elective'] = list(dict.fromkeys(blocks['elective']))
    return blocks

ai_plan = extract_courses_from_pdf('AI-curriculum.pdf')
ai_product_plan = extract_courses_from_pdf('AI_product-curriculum.pdf')
plans = {'ai': ai_plan, 'ai_product': ai_product_plan}

SELECT_PROGRAM, ASK_PROFILE, DIALOG = range(3)

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    kb = [["AI"], ["AI Product"], ["В чем разница программ?"]]
    await update.message.reply_text(
        "Здравствуйте! Я помогу подобрать подходящую магистратуру ITMO и спланировать учёбу.\n"
        "Выберите магистратуру:", reply_markup=ReplyKeyboardMarkup(kb, resize_keyboard=True)
    )
    return SELECT_PROGRAM

async def choose_program(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    t = update.message.text.lower()
    if "product" in t:
        context.user_data['prog'] = 'ai_product'
    elif t == "ai":
        context.user_data['prog'] = 'ai'
    elif "разниц" in t:
        await update.message.reply_text(
            "AI Product — про управление проектами и создание AI продуктов в бизнесе.\n"
            "AI — инженерная, техническая и исследовательская программа."
        )
        kb = [["AI"], ["AI Product"], ["В чем разница программ?"]]
        await update.message.reply_text("Выберите магистратуру:", reply_markup=ReplyKeyboardMarkup(kb, resize_keyboard=True))
        return SELECT_PROGRAM
    else:
        await update.message.reply_text('Пожалуйста, выберите "AI" или "AI Product".')
        return SELECT_PROGRAM

    await update.message.reply_text(
        'Кратко опишите ваш бэкграунд: (например, "Python, ML", "Product management", "Математика, проекты")',
        reply_markup=ReplyKeyboardMarkup([['Вернуться назад']], resize_keyboard=True)
    )
    return ASK_PROFILE

async def recommend(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    if 'назад' in update.message.text.lower():
        return await start(update, context)

    profile = update.message.text.lower()
    prog = context.user_data['prog']
    electives = plans[prog]['elective']
    if any(k in profile for k in ['продукт', 'бизнес', 'менедж', 'маркет']):
        rec = [e for e in electives if any(x in e.lower() for x in ['продукт', 'бизнес', 'управл', 'менедж', 'маркет'])]
    elif any(k in profile for k in ['python', 'разработ', 'инженер', 'backend']):
        rec = [e for e in electives if any(x in e.lower() for x in ['python', 'разработ', 'инженер', 'веб', 'микросервис'])]
    else:
        rec = [e for e in electives if any(x in e.lower() for x in ['машин', 'данные', 'stat', 'статист', 'анализ'])]
    if not rec: rec = electives[:5]

    await update.message.reply_text(f"Ваша программа: {'AI Product' if prog == 'ai_product' else 'AI'}")
    await update.message.reply_text(
        "На основании вашего профиля советую обратить внимание на такие выборные дисциплины:\n"
        + "\n".join(f"• {r}" for r in rec[:7]),
        reply_markup=ReplyKeyboardMarkup(
            [['Обязательные курсы','Выборные курсы'],
             ['В чем разница программ?','Вернуться к выбору программ']],
            resize_keyboard=True
        )
    )
    return DIALOG

async def answer_questions(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    prog = context.user_data.get('prog', 'ai')
    t = update.message.text.lower()
    if "обязательн" in t:
        man = "\n".join(f"• {d}" for d in plans[prog]['mandatory'])
        await update.message.reply_text("Обязательные дисциплины:\n" + man)
    elif "выборн" in t:
        elec = "\n".join(f"• {d}" for d in plans[prog]['elective'])
        await update.message.reply_text("Выборные дисциплины:\n" + elec)
    elif "разниц" in t:
        await update.message.reply_text(
            "AI Product — про управление проектами и создание AI продуктов в бизнесе.\n"
            "AI — инженерная, техническая и исследовательская программа."
        )
    elif "программ" in t or "магистратур" in t or "вернуться" in t:
        return await start(update, context)
    else:
        await update.message.reply_text(
            "Я могу отвечать только на вопросы о содержании учебных планов этих двух программ.\n"
            "Выберите действие:"
        )
    return DIALOG

async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text('Удачи в поступлении!')
    return ConversationHandler.END

application = ApplicationBuilder().token("YOUR_TELEGRAM_BOT_TOKEN").build()

conv = ConversationHandler(
    entry_points=[CommandHandler("start", start)],
    states={
        SELECT_PROGRAM: [MessageHandler(filters.TEXT & ~filters.COMMAND, choose_program)],
        ASK_PROFILE: [MessageHandler(filters.TEXT & ~filters.COMMAND, recommend)],
        DIALOG: [MessageHandler(filters.TEXT & ~filters.COMMAND, answer_questions)]
    },
    fallbacks=[CommandHandler("stop", cancel), CommandHandler("cancel", cancel)],
)

application.add_handler(conv)

async def main():
    await application.initialize()
    await application.start()
    await application.updater.start_polling()
    print("Бот запущен и интегрирован с вашими PDF учебных планов!")
    while True:
        await asyncio.sleep(100)

await main()
